# ICoM Simulation notebook

For this running the simulation please make sure you have:

1)list of csv with countries
2)csv contating geographic data

In this notebook we will simulate the ICoM Movment by predicting the growth in each country and cacluate the predicted cases in each country.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None

#model for testings
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR

import os
if not os.path.exists('Output'):
    os.makedirs('Output')

In [ ]:
def train(df):
    x=df.iloc[10:32,17:-1]
    y=df.iloc[10:32,-1]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

    regressor = RandomForestRegressor()
    
    #fit the model
    regressor.fit(x_train, y_train)
    # summarize feature importance
    feature_importances = pd.DataFrame(regressor.feature_importances_, index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
    feature_importances.reset_index(inplace=True)
    return regressor,feature_importances

In [ ]:
# Visualising the results for the regressor, flag is used to save the plot if true
def vizualize_prediction(regressor,y, predictions, flag):
    df = pd.DataFrame({'Actual_rf': y, 'Predicted_rf': predictions})
    df.plot(kind='bar',figsize=(10,10))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.title("Random forest",fontsize=16)
    plt.xlabel('Date ',fontsize=16)
    plt.ylabel('Growth',fontsize=16)
    if flag:
        # extract the name of the regressor
        regressor = str(regressor.__class__.__name__)
        plt.savefig('Output/'+regressor+'_predictions.png')
    else:
        plt.show()

In [ ]:
#prediction function on t+1 of weekly growth in each country
def predict(df,i,regressor):
    x=df.iloc[:33+i,17:-1]
    y=df.iloc[:33+i,-1]
    predictions=regressor.predict(x)
    return predictions

In [ ]:
#create distance from the center based on the ICoM that was created
def distFromCenter(centers, gdict, countrys):
    weekly_dist = {}
    for index in centers.index:
        center = np.array(centers['lat'][index], centers['long'][index])
        for key in gdict:
            point = np.array(gdict[key])
            distance = np.linalg.norm(center - point)
            if key in countrys:
                if key not in weekly_dist.keys():
                    weekly_dist[key] = [distance]
                else:
                    weekly_dist[key].append(distance)
    return weekly_dist

In [ ]:
#create ICom based on the caculated confirmed cases based on the prediction of the weekly growth
def ICoM(gdict, tseries, c_r, countrys):
    icom = []
    numerator = []
    denominator = []
    for country in countrys:
        coordinates = np.array([gdict[country][0], gdict[country][1]])
        numerator.append(c_r[country] * coordinates)
        denominator.append(c_r[country])
    numerator = np.sum(np.array(numerator), axis=0)
    denominator = sum(denominator)
    if denominator == 0:
        icom.append(np.array([0, 0]))
    else:
        icom.append(numerator * (1/denominator))
    icom = np.array(icom)
    icom = pd.DataFrame({'date': tseries, 'lat': icom[:, 0], 'long': icom[:, 1]})
    dist=distFromCenter(icom,gdict,countrys)
    return icom , dist

In [ ]:
#read geographic data
geo=pd.read_csv("Data/geo.csv")
geo.rename(columns = {"Unnamed: 0":"country","0":"lat","1":"long"},inplace=True)
geo=geo.set_index('country').T.to_dict('list')

In [ ]:
#create a diconery of dataframes where key is the country name
country_dict={}
predictions={}
country_list=['Germany', 'Italy', 'Spain', 'Belgium', 'Switzerland',
            'Austria','France','Finland', 'Greece', 'Netherlands']
for country in country_list:
    df=pd.read_csv("Data/"+country+".csv")
    if country not in country_dict.keys():
        country_dict[country]=df
        country_dict[country]=country_dict[country].fillna(0)

In [ ]:
#create a dictionary of trained models and extrect feature imporance for each model
feature_importance={}
models = {}
importance_df = pd.DataFrame()
for country in country_list:
    regressor,feature_importances=train(country_dict[country])
    feature_importance[country]=feature_importances
    importance_df[country]=feature_importances['index']
    # add the regressor to the country_dict dictionary
    models[country] = regressor
    
importance_df.to_csv("Output/feature importance.csv") 

In [ ]:
#create a copy of the original
original_data={}
for country in country_list:
    if country not in original_data.keys():
        original_data[country]=country_dict[country].copy()

In [ ]:
"""
Create prediction for the Weekly Growth based on a trained model 
Caculate the Weekly Confiremd Cases based on the predicted Growth for t+1
Caculate New ICoM Based on the predicted Confirmed Cases
Caculate distance from Center of ICoM for each country
"""
df_icom=pd.DataFrame(columns=['date','lat','long'])

horizon=6
time=32
pred = time
for i in range(horizon):
    predictions={}
    last_val={}
    for country in country_list:
        if country not in predictions.keys():
            predictions[country]=predict(country_dict[country],i,models[country])
        country_dict[country]["WeeklyGrowth"][time+i]=predictions[country][pred+i]
        country_dict[country]["WeeklyConfirmed"][time+i]=country_dict[country]["WeeklyGrowth"][time+i]*country_dict[country]["WeeklyConfirmed"][time-1+i]
        if country not in last_val.keys():
            last_val[country]= country_dict[country]["WeeklyConfirmed"][time+i]
    
    icom ,dist =ICoM(geo,i,last_val,country_list)
    # add the new ICoM to the df_icom dataframe
    df_icom = pd.concat([df_icom, icom], ignore_index=True)
    
    for country in country_list:
        country_dict[country]["Weeklydistance"][time+i]=dist[country][0] 

df_icom=df_icom.set_index('date')
df_icom.to_csv("Output/ICoM.csv")

In [ ]:
columns = ['country','date', 'WeeklyConfirmed', 'WeeklyConfirmed_pred', 'WeeklyGrowth', 'WeeklyGrowth_pred', 'Weeklydistance', 'Weeklydistance_pred']
results = pd.DataFrame(columns=columns)
for country in country_list:
    country_df = pd.DataFrame(columns=columns)
    country_df['date'] = country_dict[country]['date']
    country_df['WeeklyConfirmed'] = original_data[country]['WeeklyConfirmed']
    country_df['WeeklyConfirmed_pred'] = country_dict[country]['WeeklyConfirmed']
    country_df['WeeklyGrowth'] = original_data[country]['WeeklyGrowth']
    country_df['WeeklyGrowth_pred'] = country_dict[country]['WeeklyGrowth']
    country_df['Weeklydistance'] = original_data[country]['Weeklydistance']
    country_df['Weeklydistance_pred'] = country_dict[country]['Weeklydistance']
    country_df['country'] = str(country)
    results = pd.concat([results, country_df],ignore_index=True)
results.to_csv("Output/results.csv")